# Q-LoRA

### 한국어 QA 시스템 최적화

##### 0. 환경 설정

In [1]:
!pip install peft datasets transformers hf_transfer bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 77.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 18.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 73.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 57.3 MB/s  0:00:01m0:00:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 80.2 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 32.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 45.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26/26 [datasets]/26 [datasets]e]s]ub]


In [2]:
# !python -m pip install --upgrade pip

In [3]:
# !pip install typing_extensions==4.7.1 --upgrade

In [4]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq, AutoConfig
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, PeftModel
import torch.nn.functional as F

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


##### 1. 모델 로드

In [6]:
model_name = 'NCSOFT/Llama-VARCO-8B-Instruct'

In [7]:
from transformers import BitsAndBytesConfig

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [8]:
# 토크나이저 및 모델 로드

tokenizer = AutoTokenizer.from_pretrained(model_name)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

##### 2. 데이터 준비

In [10]:
# 데이터셋 로드
dataset = load_dataset('KorQuAD/squad_kor_v1')

README.md: 0.00B [00:00, ?B/s]

squad_kor_v1/train-00000-of-00001.parque(…):   0%|          | 0.00/11.6M [00:00<?, ?B/s]

squad_kor_v1/validation-00000-of-00001.p(…):   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

In [11]:
dataset['train'][0]

{'id': '6566495-0-0',
 'title': '파우스트_서곡',
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': {'text': ['교향곡'], 'answer_start': [54]}}

In [12]:
def preprocess_data(examples):
    
    inputs = ["질문: " + q + "\n문맥: " + c for q, c in zip(examples["question"], examples["context"])]
    answer_texts = [a["text"][0] if len(a["text"]) > 0 else "" for a in examples["answers"]]


    model_inputs = tokenizer(
        inputs,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )

    labels = tokenizer(
        answer_texts,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )['input_ids']
    
    max_length = model_inputs['input_ids'].shape[1]
    labels = labels[:, :max_length]


    labels[labels == tokenizer.pad_token_id] = -100

    model_inputs['labels'] = labels

    return model_inputs

In [13]:
# 데이터셋 전처리 적용
train_dataset = dataset['train'].map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names) 
val_dataset = dataset['validation'].map(preprocess_data, batched=True, remove_columns=dataset['validation'].column_names) 

Map:   0%|          | 0/60407 [00:00<?, ? examples/s]

Map:   0%|          | 0/5774 [00:00<?, ? examples/s]

In [14]:
train_dataset[0]

{'input_ids': [103194,
  52688,
  25,
  82818,
  49706,
  105078,
  16969,
  114450,
  112,
  102953,
  21028,
  56069,
  41381,
  54289,
  18918,
  118151,
  35495,
  118947,
  18359,
  108533,
  35495,
  26799,
  107762,
  16969,
  20565,
  5380,
  52688,
  127661,
  25,
  220,
  10750,
  24,
  100392,
  82818,
  49706,
  105078,
  16969,
  114450,
  112,
  102953,
  21028,
  56069,
  41381,
  54289,
  18359,
  114489,
  118151,
  35495,
  55925,
  109842,
  19954,
  109882,
  13094,
  104519,
  234,
  101103,
  117012,
  101228,
  58232,
  17835,
  61816,
  27796,
  105454,
  21028,
  101999,
  104762,
  108047,
  18359,
  108533,
  101103,
  16969,
  118183,
  18359,
  116253,
  121969,
  13,
  23955,
  45618,
  21121,
  82818,
  49706,
  105078,
  16969,
  220,
  10750,
  23,
  116899,
  120997,
  107712,
  108220,
  231,
  43139,
  105178,
  66965,
  24140,
  66965,
  18359,
  50467,
  98272,
  110,
  34804,
  116492,
  103292,
  122691,
  104834,
  54780,
  102326,
  105115,
  1

In [15]:
print(tokenizer.decode(train_dataset[0]['input_ids'], skip_special_tokens=True))

질문: 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?
문맥: 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.


##### 3. 모델 학습 준비

In [16]:
lora_config = LoraConfig(
    r=8, 
    lora_alpha=32,
    lora_dropout=0.1,
    bias='none',
    target_modules=['q_proj', 'v_proj'],
    task_type='CAUSAL_LM'
)

In [17]:
model = get_peft_model(base_model, lora_config)

model.enable_input_require_grads()
model.gradient_checkpointing_enable()
model.print_trainable_parameters()

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.0424


In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [19]:
training_args = TrainingArguments(
    output_dir='./q_lora_korqa',
    save_strategy='epoch',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    weight_decay=0.01,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
    fp16=True,
    push_to_hub=False,
    report_to='none'
)


In [20]:
trainer = Trainer(
model=model,
args=training_args,
train_dataset=train_dataset,
eval_dataset=val_dataset,
tokenizer=tokenizer,
data_collator=data_collator
)

/tmp/ipykernel_677/1766087610.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


##### 4. 모델 학습 (Fine-tuning)

In [22]:
# trainer.train()

##### 5. 학습된 모델 활용

(1) 모델 추론

In [23]:
from transformers import AutoConfig

trained_model_path = './q_lora_korqa/checkpoint-1888'

config = AutoConfig.from_pretrained(model_name)
config.save_pretrained(trained_model_path)

In [24]:
from peft import PeftModel
from transformers import pipeline

base_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype='auto', device_map='auto')
model = PeftModel.from_pretrained(base_model, trained_model_path)

qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [25]:
question = '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'
context = '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'

input_text = f"질문: {question}\n문맥: {context}"

output = qa_pipeline(input_text, max_new_tokens=50, temperature=0.2, top_p=0.8)

print(output)

[{'generated_text': '질문: 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?\n문맥: 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.)))))), ),)),)))))))))))))))))))))))))))))))))))))'}]


(2) HuggingFace Hub 

In [28]:
model.save_pretrained('./q_lora_korqa')
tokenizer.save_pretrained('./q_lora_korqa')

model.push_to_hub('VFXPEDIA/q_lora_korqa')
tokenizer.push_to_hub('VFXPEDIA/q_lora_korqa')

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/VFXPEDIA/q_lora_korqa/commit/18341aff272165a8bb584ba0059f528a54df7b1e', commit_message='Upload tokenizer', commit_description='', oid='18341aff272165a8bb584ba0059f528a54df7b1e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/VFXPEDIA/q_lora_korqa', endpoint='https://huggingface.co', repo_type='model', repo_id='VFXPEDIA/q_lora_korqa'), pr_revision=None, pr_num=None)

In [29]:
base_model = AutoModelForCausalLM.from_pretrained(
    "NCSOFT/Llama-VARCO-8B-Instruct",
    torch_dtype='auto',
    device_map='auto'
)

model = PeftModel.from_pretrained(
    base_model,
    'VFXPEDIA/q_lora_korqa'
)


tokenizer = AutoTokenizer.from_pretrained('VFXPEDIA/q_lora_korqa')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/348 [00:00<?, ?B/s]

In [30]:
inputs = tokenizer('바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?', return_tensors='pt').to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?))))))))))))))))))))))))))))))))))))))))))))))))))
